<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1


    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance


    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-07 10:40:35
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.24 C
CY Investment:  1.51 C
Reserve:  57.44 K
Current:  1.41 C
-------------------
Today PnL: 20.54 K (0.15%)
Current PnL: -20.29 L (-13.47%)
CY Booked + Current PnL: -8.85 L (-5.88%)
-------------------
Total profit:  1.50 L
Total loss:  -21.80 L
-------------------
Total Booked + Current PnL: 18.13 L (14.65%)
Total Booked PnL: 38.42 L (31.06%)
Curr Year Booked PnL: 11.44 L (8.1%)
Prev Year Booked PnL: 26.98 L (21.81%)
Est FTT:  2.29 C
Est FTT PnL: 87.80 L (62.14%)
Deployed:  1.24 C
Current:  1.41 C
CAGR/XIRR %: 8.07%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-42.00,64.0,H-MC,13.77,116054.0,15550.0,13555.0,0.84,15.47,11.68,28.96,79.0,1.15,0.82,54.69,MH,ATH,METALS
77,TTKPRESTIG,770.00,93.17,38.0,M-SC,1.35,83185.0,-17592.0,17685.0,-0.06,-17.46,21.26,0.09,245.0,-0.99,0.59,9.29,OX40N,NTT,DURABLES
14,BLUESTARCO,2080.00,9.78,48.0,H-MC,2.93,189510.0,24840.0,18496.0,0.12,15.08,9.76,26.31,89.0,1.34,1.35,24.27,X40N,NTT,AC
19,CIPLA,1662.74,-22.18,45.0,H-LC,3.91,207993.0,3493.0,19801.0,0.34,1.71,9.52,11.39,10.0,0.18,1.48,10.81,X40N,ATH,PHARMA
57,RELIANCE,1533.00,-13.66,51.0,H-LC,3.33,216091.0,5725.0,23057.0,0.74,2.72,10.67,13.68,37.0,0.25,1.54,19.69,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.5,1.24,57.0,H-LC,1.93,162585.0,-23510.0,70985.0,2.77,-12.63,43.66,25.51,15.0,-0.33,1.16,19.81,AR,ATH,ELECTRICAL
55,RAJOOENG,143.1,-38.75,48.0,H-SC,16.22,92840.0,-9660.0,50264.0,2.54,-9.42,54.14,39.61,114.0,-0.19,0.66,13.39,AR,ATH,MISC
85,WHIRLPOOL,2270.0,-46.35,35.0,M-SC,1.09,90638.0,-860.0,79616.0,2.13,-0.94,87.84,86.07,223.0,-0.01,0.64,31.62,XR,NTT,DURABLES
44,JIOFIN,387.0,-1.15,54.0,H-LC,12.75,268621.0,1256.0,64200.0,1.99,0.47,23.90,24.48,48.0,0.02,1.91,55.70,XY24,BTT,FINANCE
68,STARHEALTH,761.0,13.91,64.0,H-SC,5.76,253803.0,-27320.0,160175.0,1.98,-9.72,63.11,47.26,144.0,-0.17,1.80,35.98,XY24,NTT,INSURANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.0,1222.17,63.0,M-SC,20.84,179868.0,-1275.0,46748.0,-2.06,-0.70,25.99,25.10,235.0,-0.03,1.28,44.15,XY24,NTT,HEALTHCARE
38,INDIGOPNTS,1408.0,101.45,36.0,M-SC,2.10,132072.0,-42487.0,42514.0,-1.49,-24.34,32.19,0.02,221.0,-1.00,0.94,14.88,OX40N,NTT,PAINTS
78,UJJIVANSFB,60.0,106.53,63.0,H-SC,13.47,130248.0,-12231.0,31754.0,-1.33,-8.58,24.38,13.70,163.0,-0.39,0.93,53.73,OX40N,NTT,BANKS
16,CAMPUS,393.0,-25.51,51.0,M-SC,1.76,155081.0,-12694.0,68530.0,-1.32,-7.57,44.19,33.28,210.0,-0.19,1.10,20.20,XY24,NTT,FOOTWEAR
69,SURYODAY,240.0,62.28,63.0,H-SC,6.68,149165.0,-29906.0,95630.0,-1.32,-16.70,64.11,36.71,135.0,-0.31,1.06,48.00,XR,NTT,BANKS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.0,1222.17,63.0,M-SC,20.84,179868.0,-1275.0,46748.0,-2.06,-0.70,25.99,25.10,235.0,-0.03,1.28,44.15,XY24,NTT,HEALTHCARE
85,WHIRLPOOL,2270.0,-46.35,35.0,M-SC,1.09,90638.0,-860.0,79616.0,2.13,-0.94,87.84,86.07,223.0,-0.01,0.64,31.62,XR,NTT,DURABLES
35,ICICIPRULI,790.0,-20.77,50.0,H-MC,2.05,136312.0,524.0,42229.0,0.45,0.39,30.98,31.48,107.0,0.01,0.97,12.55,X40,ATH,INSURANCE
86,WIPRO,420.0,-14.25,46.0,M-LC,5.70,151032.0,87.0,109362.0,0.61,0.06,72.41,72.51,53.0,0.00,1.07,6.08,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,106.53,63.0,H-SC,13.47,130248.0,-12231.0,31754.0,-1.33,-8.58,24.38,13.70,163.0,-0.39,0.93,53.73,OX40N,NTT,BANKS
45,KANSAINER,340.0,-67.42,56.0,H-SC,2.11,225000.0,-44667.0,81000.0,-0.39,-16.56,36.00,13.47,138.0,-0.55,1.60,14.41,XY24,NTT,PAINTS
77,TTKPRESTIG,770.0,93.17,38.0,M-SC,1.35,83185.0,-17592.0,17685.0,-0.06,-17.46,21.26,0.09,245.0,-0.99,0.59,9.29,OX40N,NTT,DURABLES
18,CERA,9475.0,-22.20,41.0,H-SC,1.50,141634.0,-34269.0,76284.0,0.57,-19.48,53.86,23.89,149.0,-0.45,1.01,22.32,OX40N,NTT,CERAMICS
72,TATAELXSI,9161.0,-24.89,40.0,H-MC,6.26,101526.0,-30309.0,72530.0,-0.94,-22.99,71.44,32.03,98.0,-0.42,0.72,13.72,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,44.35,52.0,M-MC,12.36,227597.0,2635.0,165031.0,-0.34,1.17,72.51,74.53,192.0,0.02,1.62,33.80,XY24,BTT,STEEL
32,HINDZINC,730.22,31.07,68.0,M-LC,6.24,213520.0,8444.0,104134.0,-0.13,4.12,48.77,54.89,52.0,0.08,1.52,29.21,X5K,ATH,METALS
14,BLUESTARCO,2080.00,9.78,48.0,H-MC,2.93,189510.0,24840.0,18496.0,0.12,15.08,9.76,26.31,89.0,1.34,1.35,24.27,X40N,NTT,AC
84,VOLTAS,1530.00,-2.20,48.0,H-MC,1.43,205305.0,13563.0,24185.0,0.15,7.07,11.78,19.69,99.0,0.56,1.46,14.35,XY25,NTT,AC
17,CAMS,5211.76,-5.16,49.0,H-SC,1.80,107918.0,5914.0,38009.0,0.75,5.80,35.22,43.06,122.0,0.16,0.77,23.91,X40N,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-42.00,64.0,H-MC,13.77,116054.0,15550.0,13555.0,0.84,15.47,11.68,28.96,79.0,1.15,0.82,54.69,MH,ATH,METALS
11,BANKINDIA,190.00,-24.22,72.0,H-MC,10.87,195880.0,16072.0,96334.0,1.05,8.94,49.18,62.52,88.0,0.17,1.39,46.07,XR,NTT,BANKS
32,HINDZINC,730.22,31.07,68.0,M-LC,6.24,213520.0,8444.0,104134.0,-0.13,4.12,48.77,54.89,52.0,0.08,1.52,29.21,X5K,ATH,METALS
37,INDIAMART,4810.62,-54.83,36.0,H-SC,9.35,126235.0,2899.0,128722.0,0.24,2.35,101.97,106.72,119.0,0.02,0.90,26.28,AR,ATH,MISC
86,WIPRO,420.00,-14.25,46.0,M-LC,5.70,151032.0,87.0,109362.0,0.61,0.06,72.41,72.51,53.0,0.00,1.07,6.08,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-22.15,30.0,H-LC,9.96,280305.0,-35537.0,148197.0,0.09,-11.25,52.87,35.67,5.0,-0.24,1.99,1.04,X40N,ATH,FMCG
5,ASIANPAINT,3465.66,-17.64,35.0,H-LC,5.77,207777.0,-43991.0,97198.0,0.27,-17.47,46.78,21.13,27.0,-0.45,1.48,11.76,X40,ATH,PAINTS
42,ITC,452.00,-39.19,42.0,H-LC,2.14,196138.0,-4000.0,24890.0,0.09,-2.00,12.69,10.44,4.0,-0.16,1.39,3.45,X40,NTT,FMCG
74,TCS,4406.34,-29.97,43.0,H-LC,12.19,279142.0,-66814.0,135049.0,-0.63,-19.31,48.38,19.73,1.0,-0.49,1.98,2.81,X40,ATH,IT
19,CIPLA,1662.74,-22.18,45.0,H-LC,3.91,207993.0,3493.0,19801.0,0.34,1.71,9.52,11.39,10.0,0.18,1.48,10.81,X40N,ATH,PHARMA


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.5,1.24,57.0,H-LC,1.93,162585.0,-23510.0,70985.0,2.77,-12.63,43.66,25.51,15.0,-0.33,1.16,19.81,AR,ATH,ELECTRICAL
42,ITC,452.0,-39.19,42.0,H-LC,2.14,196138.0,-4000.0,24890.0,0.09,-2.00,12.69,10.44,4.0,-0.16,1.39,3.45,X40,NTT,FMCG
48,LTIM,7230.2,-4.34,54.0,H-LC,2.37,237532.0,-8619.0,87816.0,0.09,-3.50,36.97,32.18,16.0,-0.10,1.69,31.28,X200,ATH,IT
51,NESTLEIND,1377.0,-9.88,53.0,H-LC,3.07,277407.0,11981.0,44801.0,0.32,4.51,16.15,21.40,11.0,0.27,1.97,11.46,XY25,NTT,FMCG
57,RELIANCE,1533.0,-13.66,51.0,H-LC,3.33,216091.0,5725.0,23057.0,0.74,2.72,10.67,13.68,37.0,0.25,1.54,19.69,XY25,NTT,REFINERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-22.15,30.0,H-LC,9.96,280305.0,-35537.0,148197.0,0.09,-11.25,52.87,35.67,5.0,-0.24,1.99,1.04,X40N,ATH,FMCG
74,TCS,4406.34,-29.97,43.0,H-LC,12.19,279142.0,-66814.0,135049.0,-0.63,-19.31,48.38,19.73,1.0,-0.49,1.98,2.81,X40,ATH,IT
42,ITC,452.00,-39.19,42.0,H-LC,2.14,196138.0,-4000.0,24890.0,0.09,-2.00,12.69,10.44,4.0,-0.16,1.39,3.45,X40,NTT,FMCG
30,HCLTECH,1937.36,-1.51,48.0,H-LC,9.87,224432.0,-17492.0,79741.0,0.83,-7.23,35.53,25.73,8.0,-0.22,1.59,6.06,X40,ATH,IT
80,UNITDSPR,1644.00,-11.37,56.0,H-LC,6.88,232286.0,-1858.0,50476.0,-0.77,-0.79,21.73,20.77,86.0,-0.04,1.65,6.07,X40N,NTT,BREWERIES


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,1.24,57.0,H-LC,1.93,162585.0,-23510.0,70985.0,2.77,-12.63,43.66,25.51,15.0,-0.33,1.16,19.81,AR,ATH,ELECTRICAL
42,ITC,452.00,-39.19,42.0,H-LC,2.14,196138.0,-4000.0,24890.0,0.09,-2.00,12.69,10.44,4.0,-0.16,1.39,3.45,X40,NTT,FMCG
19,CIPLA,1662.74,-22.18,45.0,H-LC,3.91,207993.0,3493.0,19801.0,0.34,1.71,9.52,11.39,10.0,0.18,1.48,10.81,X40N,ATH,PHARMA
5,ASIANPAINT,3465.66,-17.64,35.0,H-LC,5.77,207777.0,-43991.0,97198.0,0.27,-17.47,46.78,21.13,27.0,-0.45,1.48,11.76,X40,ATH,PAINTS
57,RELIANCE,1533.00,-13.66,51.0,H-LC,3.33,216091.0,5725.0,23057.0,0.74,2.72,10.67,13.68,37.0,0.25,1.54,19.69,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4406.34,-29.97,43.0,H-LC,12.19,279142.0,-66814.0,135049.0,-0.63,-19.31,48.38,19.73,1.0,-0.49,1.98,2.81,X40,ATH,IT
40,INFY,2275.00,-19.96,46.0,H-LC,8.08,312024.0,-864.0,172549.0,-0.75,-0.28,55.30,54.87,3.0,-0.01,2.22,6.35,X40,BTT,IT
42,ITC,452.00,-39.19,42.0,H-LC,2.14,196138.0,-4000.0,24890.0,0.09,-2.00,12.69,10.44,4.0,-0.16,1.39,3.45,X40,NTT,FMCG
83,VBL,671.64,-22.15,30.0,H-LC,9.96,280305.0,-35537.0,148197.0,0.09,-11.25,52.87,35.67,5.0,-0.24,1.99,1.04,X40N,ATH,FMCG
1,ABB,7934.00,-39.48,54.0,H-LC,7.50,253104.0,-8515.0,127716.0,1.05,-3.25,50.46,45.57,7.0,-0.07,1.80,7.70,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-45.67,45.0,L-SC,31.66,79680.0,-33869.0,73919.0,0.65,-29.83,92.77,35.27,268.0,-0.46,0.57,95.02,XR,NTT,HOTELS
49,MASFIN,398.61,-19.15,46.0,H-SC,11.47,92040.0,-5940.0,27548.0,-0.44,-6.06,29.93,22.05,152.0,-0.22,0.65,33.79,XR,ATH,FINANCE
50,NATIONALUM,244.55,-42.00,64.0,H-MC,13.77,116054.0,15550.0,13555.0,0.84,15.47,11.68,28.96,79.0,1.15,0.82,54.69,MH,ATH,METALS
78,UJJIVANSFB,60.00,106.53,63.0,H-SC,13.47,130248.0,-12231.0,31754.0,-1.33,-8.58,24.38,13.70,163.0,-0.39,0.93,53.73,OX40N,NTT,BANKS
61,SAMMAANCAP,326.00,-184.09,68.0,M-SC,12.55,145809.0,-3411.0,147588.0,1.29,-2.29,101.22,96.62,208.0,-0.02,1.04,57.80,XY25,NTT,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAMMAANCAP,326.00,-184.09,68.0,M-SC,12.55,145809.0,-3411.0,147588.0,1.29,-2.29,101.22,96.62,208.0,-0.02,1.04,57.80,XY25,NTT,FINANCE
69,SURYODAY,240.00,62.28,63.0,H-SC,6.68,149165.0,-29906.0,95630.0,-1.32,-16.70,64.11,36.71,135.0,-0.31,1.06,48.00,XR,NTT,BANKS
32,HINDZINC,730.22,31.07,68.0,M-LC,6.24,213520.0,8444.0,104134.0,-0.13,4.12,48.77,54.89,52.0,0.08,1.52,29.21,X5K,ATH,METALS
26,GREENPANEL,537.00,232.04,59.0,M-SC,3.68,148181.0,-34897.0,113877.0,0.48,-19.06,76.85,43.14,230.0,-0.31,1.05,39.38,XY24,NTT,MISC
11,BANKINDIA,190.00,-24.22,72.0,H-MC,10.87,195880.0,16072.0,96334.0,1.05,8.94,49.18,62.52,88.0,0.17,1.39,46.07,XR,NTT,BANKS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.10
1,25,43.27
2,50,74.96


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.06
LC    32.21
MC    23.71
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.90
X40      14.56
X40N     11.19
XR       10.99
XY25     10.42
AR        8.23
OX40N     7.72
X200      1.69
MH        1.61
X5K       1.52
SR        1.15
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    28.98
H-LC    25.66
H-MC    20.48
M-SC    13.66
M-LC     5.54
M-MC     2.93
L-SC     1.42
L-LC     1.01
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.77,-5.88,40.78
IT,12.60,-18.48,82.64
FINANCE,10.50,-14.14,69.26
MISC,6.96,-17.18,82.10
BANKS,6.37,-11.37,69.95
PAINTS,5.55,-18.28,35.91
ELECTRICAL,5.39,-8.69,46.82
INSURANCE,3.78,-3.82,42.63
AUTO,3.52,-15.92,71.34


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3222545.0,22
XR,1361131.0,14
AR,1173334.0,9
X40,789880.0,10
X40N,660135.0,9
OX40N,561378.0,10
XY25,472171.0,7
SR,260366.0,2
X5K,104134.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3712360.0,29
M-SC,1507215.0,17
H-MC,1270686.0,15
H-LC,1184256.0,15
M-MC,371735.0,2
M-LC,370943.0,4
L-SC,261154.0,3
L-MC,60240.0,1
L-LC,41487.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1209674.0      6
M-SC       XY24       850180.0      7
H-SC       AR         806827.0      5
           XR         795041.0      7
H-MC       XY24       569309.0      4
H-LC       X40        509427.0      5
M-MC       XY24       371735.0      2
H-SC       X40N       308365.0      4
           SR         260366.0      2
           OX40N      258456.0      4
H-LC       X40N       218474.0      3
H-MC       X40        208204.0      4
H-LC       AR         198701.0      2
H-MC       XY25       177458.0      2
L-SC       XR         172197.0      2
M-SC       AR         167806.0      2
M-LC       XY24       157447.0      2
M-SC       XY25       147588.0      1
           OX40N      141435.0      4
H-MC       X40N       133296.0      2
M-SC       XR         127957.0      2
M-LC       XR         109362.0      1
H-LC       XY25       105638.0      3
M-LC       X5K        104134.0      1
H-MC       XR          96334.0      1
L-SC       OX40N       88957.0      1
H-LC       X200        87816.0      1
H-SC       MH          73631.0      1
H-MC       OX40N       72530.0      1
M-SC       X40         72249.0      1
H-LC       XY24        64200.0      1
L-MC       XR          60240.0      1
L-LC       XY25        41487.0      1
H-MC       MH          13555.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 37.0 seconds
